In [214]:
import rospy
import tf
import json
from moveit_msgs.srv import GetPositionIK, GetPositionIKRequest
from geometry_msgs.msg import PoseStamped
from cs_golf.robot import Robot
from cs_golf.trajectories import trapezoidal_speed_trajectory
from copy import deepcopy
from cs_golf.transformations import quat_rotate, multiply_transform, inverse_transform, list_to_pose_stamped
from moveit_msgs.msg import RobotTrajectory
from trajectory_msgs.msg import JointTrajectoryPoint

In [5]:
rospy.init_node("putter_rotations")

In [73]:
robot = Robot()

In [6]:
tfl = tf.TransformListener()

In [13]:
tfb = tf.TransformBroadcaster()

In [12]:
# rosrun tf static_transform_publisher -0.385 -0.396 0.02 -0.604 -0.405 -0.601 0.331 iiwa_link_ee putter 50

In [84]:
tfl.lookupTransform("world", "iiwa_link_ee", rospy.Time(0))

([0.12532283301822617, -0.3851893791795562, 0.7587413204452541],
 [0.6154466337612458,
  0.40159065654929116,
  0.6013661089366799,
  0.31354296130931947])

In [85]:
req = GetPositionIKRequest()
req.ik_request.group_name = "manipulator"
req.ik_request.ik_link_name = "iiwa_link_ee"

In [105]:
p = PoseStamped()
p.header.frame_id = "world"
p.pose.position.x = 0.12
p.pose.position.y = -0.38
p.pose.position.z = 0.9
p.pose.orientation.x = 0.6154466337612458
p.pose.orientation.y = 0.40159065654929116
p.pose.orientation.z = 0.6013661089366799
p.pose.orientation.w = 0.31354296130931947

In [106]:
req.ik_request.pose_stamped = p
req.ik_request.robot_state = robot.current_state

In [107]:
req

ik_request: 
  group_name: "manipulator"
  robot_state: 
    joint_state: 
      header: 
        seq: 0
        stamp: 
          secs: 0
          nsecs:         0
        frame_id: "world"
      name: [iiwa_joint_1, iiwa_joint_2, iiwa_joint_3, iiwa_joint_4, iiwa_joint_5, iiwa_joint_6,
  iiwa_joint_7]
      position: [-1.5531087493483133, -0.0038530481978972498, -0.01682261604940589, -1.502484686923476, 1.4973296726054919, 1.6726883641467405, 0.576573583704487]
      velocity: []
      effort: []
    multi_dof_joint_state: 
      header: 
        seq: 0
        stamp: 
          secs: 0
          nsecs:         0
        frame_id: "world"
      joint_names: []
      transforms: []
      twist: []
      wrench: []
    attached_collision_objects: []
    is_diff: False
  constraints: 
    name: ''
    joint_constraints: []
    position_constraints: []
    orientation_constraints: []
    visibility_constraints: []
  avoid_collisions: False
  ik_link_name: "iiwa_link_ee"
  pose_stamped: 


In [108]:
ik = rospy.ServiceProxy("/iiwa/compute_ik", GetPositionIK)

In [109]:
result = ik(req)

In [110]:
if result.error_code.val == -31:
    print("No IK solution")

In [111]:
traj = trapezoidal_speed_trajectory(result.solution, robot.current_state, 1, 1)

In [112]:
robot.display(traj)

In [203]:
DISTANCE_TO_CENTRE = 0.075   # Run-up
TRAJECTORY_RANGE = 0.2       # In-line trajectory distance

In [135]:
base_putter = [[0.111, -0.603, 0.029],  [0,0,0,1]]
world_P_centre = deepcopy(base_putter)
world_P_centre[0][0] += DISTANCE_TO_CENTRE

In [136]:
world_P_centre

[[0.186, -0.603, 0.029], [0, 0, 0, 1]]

In [149]:
gripper_T_putter = [[-0.385, -0.396, 0.02], [-0.604, -0.405, -0.601, 0.331]]
putter_T_gripper = inverse_transform(gripper_T_putter)

In [141]:
rotation = tf.transformations.quaternion_about_axis(0.5, (0, 0, 1))
position = quat_rotate(rotation, [-DISTANCE_TO_CENTRE, 0, 0])
centre_T_putter = [position, rotation]

In [142]:
world_P_centre = ball_rot

In [144]:
world_P_putter = multiply_transform(world_P_centre, centre_T_putter)

In [145]:
tfb.sendTransform(world_P_putter[0], world_P_putter[1], rospy.Time.now(), "putter2", "world")

In [206]:
NUM_ROTATIONS = 100
NUM_POINTS_IN_TRAJ = 20

# Seed is poses["init"]
last_ik_seed = robot.current_state
last_ik_seed.joint_state.position = [-1.601099757776293, 0.23286978848422024, -0.03592373282101929,
                                     -1.7601794976359955, 1.456285140355729, 1.6949230560747754, 0.05387100838806534]

trajectories = []
for rot in range(NUM_ROTATIONS):
    # Compute each putter angle between [-1, 0.86[ rad
    angle = -1 + rot*1.85/NUM_ROTATIONS
    rotation = tf.transformations.quaternion_about_axis(angle, (0, 0, 1))
    trajectory = {"angle": angle, "points": []}
    last_ik_seed_point = last_ik_seed
    for point in range(NUM_POINTS_IN_TRAJ):
        distance_to_point = float(point)*TRAJECTORY_RANGE/NUM_POINTS_IN_TRAJ
        position = quat_rotate(rotation, [-DISTANCE_TO_CENTRE + distance_to_point, 0, 0])
        centre_T_putter = [position, rotation]
        world_P_putter = multiply_transform(world_P_centre, centre_T_putter)
        tfb.sendTransform(world_P_putter[0], world_P_putter[1], rospy.Time.now(), "putter2", "world")

        # Get end effector pose for each angle
        world_P_gripper = multiply_transform(world_P_putter, putter_T_gripper)
        tfb.sendTransform(world_P_gripper[0], world_P_gripper[1], rospy.Time.now(), "gripper", "world")
        ps_eef = list_to_pose_stamped(world_P_gripper, frame_id="world")

        # Get IK for the init pose
        # /!!!!\ The seed is the current pose
        req = GetPositionIKRequest()
        req.ik_request.group_name = "manipulator"
        req.ik_request.ik_link_name = "iiwa_link_ee"
        req.ik_request.pose_stamped = ps_eef
        req.ik_request.robot_state = last_ik_seed_point
        result = ik(req)
        if result.error_code.val == -31:
            print("No IK solution for", rot, "angle", angle, "point", point, "distance", distance_to_point)
        else:
            #print(i, angle, j)
            last_ik_seed_point = result.solution
            traj = trapezoidal_speed_trajectory(result.solution, last_ik_seed_point, 1, 1)
            #robot.display(traj)
            #rospy.sleep(0.5)
            trajectory["points"].append(result.solution.joint_state.position)
            if point == 0:
                last_ik_seed = last_ik_seed_point
        #rospy.sleep(0.05)
    trajectories.append(trajectory)

In [212]:
for t in trajectories:
    rt = RobotTrajectory()
    rt.joint_trajectory.joint_names = last_ik_seed.joint_state.name
    for point in t["points"]:
        jtp = JointTrajectoryPoint()
        jtp.positions = point
        rt.joint_trajectory.points.append(jtp)
    robot.display(rt)
    rospy.sleep(0.5)

In [215]:
with open("../config/motions.json", "w") as f:
    json.dump({"joints": last_ik_seed.joint_state.name, "trajectories": trajectories}, f)